# Final Project : 3D reconstruction with multi-images

## CS445: Computational Photography - Fall 2019

In [1]:
import numpy as np
import scipy
def sym_params(i,j):
    """ params of Q for iQQTj"""
    o=np.outer(i,j)
    o=o+o.T-np.diag(o.diagonal())
    return o[np.triu_indices(3)]
def tomasi_kanade(W0):
    """ Tomasi–Kanade factorization
        input:
        W0 - coordinates of shape (Frames,Points,2)
        output:
        R - (2*Frames,3)
        S - (3,Points)
        """
    F,P,_=W0.shape
    # (2F,P) 2F stack Xs and Ys
    W=W0.transpose(2,0,1).reshape(F*2,P).astype(np.float64)
    # normalize to centroid
    W-=W.mean(axis=1,keepdims=True)
    # svd, (2F,P), (,P), (P,P)
    # svd 2F<P, (2F,2F), (,2F), (2F,P)
    u, s, vh =np.linalg.svd(W,full_matrices=True)
    # (2F,3),(3,P)
    R_hat=u[:,:3]*(s[:3]**0.5)
    S_hat=((s[:3]**0.5)*vh[:3,:].T).T
    # (F,3) (F,3)
    I,J=R_hat[:F,:],R_hat[F:,:]
    # solve Q*QT, symmetric, 6 params
    # iTQi=1 jTQj=1 iTQj=0 R=[i,j,k]T
    A=np.zeros([3*F,6])
    A[:F,:]=np.array([sym_params(I[i],I[i]) for i in range(len(I))])
    A[F:2*F,:]=np.array([sym_params(J[i],J[i]) for i in range(len(I))])
    A[2*F:,:]=np.array([sym_params(I[i],J[i]) for i in range(len(I))])
    # ii,jj are 1, ij are 0
    b=np.ones([3*F])
    b[2*F:]=0
    # solve QQT
    qsym=np.linalg.lstsq(A,b)[0]
    # get symmetric mat
    QSym=np.zeros([3,3])
    QSym[np.triu_indices(3)]=qsym
    QSym[np.tril_indices(3,k=-1)]=QSym[np.triu_indices(3,k=1)]
    try:
        Q=np.linalg.cholesky(QSym).T
    except:
        print("Bad Data")
        return None,None
    # get R,S
    R=R_hat.dot(Q)
    S=np.linalg.inv(Q).dot(S_hat)
    return R,S

In [2]:
def removeNaN(a):
    b=[]
    for i in range(a.shape[0]):
        if (a[i].max()>-1e10):
            b.append(a[i])
    return np.array(b)

In [5]:
import scipy.io
matdata=scipy.io.loadmat("tracked_points")
Xs=matdata['Xs']
Ys=matdata['Ys']
Wmat=removeNaN(np.stack((Xs,Ys)).transpose([2,1,0])).transpose([1,0,2])

In [6]:
R,S=tomasi_kanade(Wm2)

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [238]:
from plotly.offline import plot
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
tripoints3d=S # (3,P)
fig = go.Figure(data=[go.Scatter3d(x=tripoints3d[0],
                                   y=tripoints3d[1],
                                   z=tripoints3d[2],
                                   mode='markers')])
plot(fig, filename='3d-axis-range')

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `3d-axis-range` didn't end with .html. Adding .html to the end of your file.



'3d-axis-range.html'